In [12]:
from PIL import Image
from PIL import ImageEnhance
import cv2  
import numpy

##### 分类别读取数据

In [13]:
datas=[]
    
for i in range(12):
    datas.append([])
    
def classification(label_path,data_path,datas):
    file=open(label_path,'r')
    lines=file.readlines()

    labels=[]
    
    for line in lines:
        line=line.split('\t')
        img_path=data_path+'/'+line[0]
        label=int(line[1])
        datas[label].append(img_path)
    return datas


datas=classification('./E3_data/Test_labels.txt','./E3_data/test_img',datas)
datas=classification('./E3_data/train_labels.txt','./E3_data/train_img',datas)
datas=classification('./E3_data/val_labels.txt','./E3_data/val_img',datas)

##### 定义计算面积函数

In [14]:
def areaCal(img_path):
    
    img = Image.open(img_path)
    enh_con = ImageEnhance.Contrast(img)  
    contrast = 1000
    img_contrasted = enh_con.enhance(contrast)   
    img_contrasted.save("./transition.png")
    
    
    img_path = r'./transition.png'
    img = cv2.imread(img_path)
    red = img[:, :, 2]
    
    ret, binary = cv2.threshold(red, 0, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    area1 = 0
    for i in range(len(contours)):
        area1 += cv2.contourArea(contours[i])
        
    ret, binary = cv2.threshold(red, 100, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    area2 = 0
    for i in range(len(contours)):
        area2 += cv2.contourArea(contours[i])

    return (area1+area2)/2


##### 计算每一张图片的面积

In [4]:
results=[]
    
for i in range(12):
    results.append([])

for i in range(12):
    for j in datas[i]:
        results[i].append(areaCal(j))
        

In [5]:
results

[[664.25,
  700.25,
  1316.25,
  300.0,
  1396.25,
  259.25,
  1018.75,
  1215.25,
  1510.75,
  1316.0,
  1209.25,
  1061.25,
  2247.75,
  1789.5,
  1200.25,
  1482.75,
  1021.0,
  1468.5,
  1326.0,
  1419.0,
  787.0,
  1997.75,
  369.75,
  1793.5,
  681.5,
  1485.75,
  1394.5,
  1350.25,
  1572.0,
  1536.0,
  1007.75,
  1378.25,
  1635.75,
  1293.5,
  1249.75,
  1399.5,
  1273.75,
  1842.5,
  1165.5,
  1607.5,
  965.75,
  1115.0,
  1183.0,
  1639.0,
  1444.0,
  1753.5,
  1307.0,
  712.25,
  2049.75,
  961.75,
  269.25,
  376.0,
  1368.0,
  1263.5,
  1208.0,
  1240.75,
  1308.25,
  4488.0,
  1037.5,
  749.5,
  997.75,
  984.0,
  2210.0,
  1336.75,
  600.5,
  1302.0,
  92.5,
  0.0,
  1651.25,
  1987.75,
  997.25,
  1685.0,
  1996.25,
  1822.75,
  1082.25,
  2173.0,
  1446.5,
  1255.75,
  1553.25,
  1291.75,
  1739.75,
  1303.0,
  1665.75,
  0.0,
  63.0,
  1767.75,
  2453.0,
  282.0,
  1636.25,
  2022.5,
  1330.25,
  1728.25,
  1803.25,
  2174.0,
  1862.0,
  2309.25,
  904.25,
  38.5,
  

##### 为每一张图片提供标注，0代表面积计算无效，1代表面积计算有效

In [7]:
mask=[]
    
for i in range(12):
    mask.append([])

def test_mask(path):
    
    img = cv2.imread(path)
    red = img[:, :, 2]
    
    ret, binary = cv2.threshold(red, 10, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    area = 0
    for i in range(len(contours)):
        area += cv2.contourArea(contours[i])
    
    return area==0


for i in range(12):
    for j in range(len(datas[i])):
        if test_mask(datas[i][j]):
            mask[i].append(0)
        else:
            mask[i].append(1)
        



    


##### 将结果写入函数

In [11]:
file_name = 'results.txt'

with open(file_name,'w') as file_obj:
    for i in range(len(datas)):
        for j in range(len(datas[i])):
            file_obj.write(str(datas[i][j]))
            file_obj.write("\t")
            file_obj.write(str(results[i][j]))
            file_obj.write("\t")
            file_obj.write(str(mask[i][j]))
            file_obj.write("\n")